#### In notebook moranSquirrels, I had to run simulations to estimate probabilities for fixation. Now, to get a better sense of what is going on (and much more efficiently) I'm going to numerically compute fixation probabilities.

In [1]:
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [2]:
def survival_matrix(N):
    surv_mat = zero_matrix(QQ, N)
    for i in range(N):
        surv_mat[i,i] = 1 - 1/N
        
    return surv_mat

def age_up_matrix(N):
    age_up_mat = zero_matrix(QQ, N)
    
    age_up_mat[0, 0] = 1
    
    for i in range(1, N - 1):
        age_up_mat[i+1, i] = 1
    
    age_up_mat[-1, -1] = 1
    
    return age_up_mat

In [3]:
def num_fertile_A(current_pop):
    return current_pop[0]

def num_fertile_B(current_pop):
    return current_pop[-1]

def A_reproduction_probability(current_pop, beta):
    return num_fertile_A(current_pop)/(num_fertile_A(current_pop) + beta*num_fertile_B(current_pop))

def kill_someone(current_pop, surv_mat):
    return surv_mat*current_pop

def birth_someone_and_age_up(current_pop, A_rep_prob, age_up_mat):
    current_pop = age_up_mat*current_pop
    current_pop[0] += A_rep_prob
    current_pop[1] += 1 - A_rep_prob
    
    return current_pop


In [4]:
def expected_day_of_squirrels(current_pop, beta, surv_mat = None, age_up_mat = None):
    
    N = len(current_pop)
    if surv_mat == None:
        surv_mat = survival_matrix(N)
    if age_up_mat == None:
        age_up_mat = age_up_matrix(N)
    
    p = A_reproduction_probability(current_pop, beta)
    current_pop = kill_someone(current_pop, surv_mat)
    current_pop = birth_someone_and_age_up(current_pop, p, age_up_mat)
    
    return current_pop.n()

In [45]:
current_pop = vector((10, 1, 0, 0, 0, 0, 0, 0,0, 0, 0))
for i in tqdm(range(20000)):
    current_pop = expected_day_of_squirrels(current_pop, 2.42)
current_pop

  0%|          | 0/20000 [00:00<?, ?it/s]

(1.22819536627093e-17, 1.00000000000000, 0.909090909090909, 0.826446280991735, 0.751314800901578, 0.683013455365071, 0.620921323059155, 0.564473930053777, 0.513158118230707, 0.466507380209733, 4.66507380209732)

#### Naturally, indefinite coexistence of the two types is impossible. It seems there is a breaking point at which B's go from being whiped out to fixating. This seems another natural measure of fitness... A's and B's have equivalent fitness if B is just as likely to be whiped out as it is to whipe out. This is likely asymmetric.

#### It is also important to note that none of this work answers the question I wanted to ask: "What is $P(B \text{ fixate } )$?" 

In [22]:
def gen_combinations(n, k):
    """
    Returns all 'combinations' (combinatorial term meaning ways of summing up to a number)
    of n by k parts. For instance, gen_combinations(2, 2) = [[2, 0], [1, 1], [0, 2]].
    """
    min_elem = 0
    max_elem = n
    allowed = range(max_elem, min_elem-1, -1)

    def helper(n, k, t):
        if k == 0:
            if n == 0:
                yield t
        elif k == 1:
            if n in allowed:
                yield t + [n,]
        elif min_elem * k <= n <= max_elem * k:
            for v in allowed:
                yield from helper(n - v, k - 1, t + [v,])

    return list(helper(n, k, []))

In [23]:
def delta(i, j):
    if i == j:
        return 1
    else:
        return 0

In [24]:
def succ_A_die_A_rep(c1):
    list_of_successors = []
    if len(c1) == 2:
        succ = c1
    else:
        succ = [c1[0]] + [0] + [c1[i] for i in range(1, len(c1) - 2)] + [c1[-2] + c1[-1]]
    list_of_successors.append(succ)
    
    return list_of_successors

def succ_A_die_B_rep(c1):
    list_of_successors = []
    if len(c1) == 2:
        succ = [c1[0] -1, c1[1] + 1]
    else:
        succ = [c1[0] - 1] + [1] + [c1[i] for i in range(1, len(c1) - 2)] + [c1[-2] + c1[-1]]
    list_of_successors.append(succ)
    
    return list_of_successors
    
def succ_B_die_A_rep(c1):
    list_of_successors = []
    if len(c1) == 2:
        succ = [c1[0] +1, c1[1] -1]
        list_of_successors.append(succ)
    else:
        for a in range(1, len(c1) - 1):
            succ = [c1[0] + 1] + [0] + [max(c1[i] - delta(a, i), 0) for i in range(1, len(c1) - 2)] + [c1[-2] + c1[-1]]
            if sum(succ) == sum(c1):
                list_of_successors.append(succ)
        succ = [c1[0] + 1] + [0] + [c1[i] for i in range(1, len(c1) - 2)] + [max(c1[-2] + c1[-1] - 1, 0)]
        if sum(succ) == sum(c1):
            list_of_successors.append(succ)
        
    return list_of_successors

def succ_B_die_B_rep(c1):
    list_of_successors = []
    if len(c1) == 2:
        succ = c1
        list_of_successors.append(succ)
    else:
    
        for a in range(1, len(c1) - 1):
            succ = [c1[0]] + [1] + [max(c1[i] - delta(a, i), 0) for i in range(1, len(c1) - 2)] + [c1[-2] + c1[-1]]
            if sum(succ) == sum(c1):
                list_of_successors.append(succ)
        succ = [c1[0]] + [1] + [c1[i] for i in range(1, len(c1) - 2)] + [max(c1[-2] + c1[-1] - 1, 0)]
        if sum(succ) == sum(c1):
            list_of_successors.append(succ)
    
    return list_of_successors

def all_successors(c1):
    s1 = succ_A_die_A_rep(c1)
    s2 = succ_A_die_B_rep(c1)
    s3 = succ_B_die_A_rep(c1)
    s4 = succ_B_die_B_rep(c1)
    
    return s1 + s2 + s3 + s4

In [25]:
def A_died_A_reproduced(c1, c2):
    return c2 in succ_A_die_A_rep(c1)

def A_died_B_reproduced(c1, c2):
    return c2 in succ_A_die_B_rep(c1)

def B_died_A_reproduced(c1, c2):
    return c2 in succ_B_die_A_rep(c1)

def B_died_B_reproduced(c1, c2):
    return c2 in succ_B_die_B_rep(c1)

In [26]:
def age_at_which_death_occurred(c1, c2):
    if A_died_A_reproduced(c1,c2) or A_died_B_reproduced(c1, c2):
        return None
    else:
        for a in range(1, len(c1) - 2):
            if c1[a] != c2[a + 1]:
                return a
        return len(c2) - 2

In [27]:
def prob_A_die(c1):
    return c1[0]/sum(c1)

def prob_B_die(c1):
    return 1 - prob_A_die(c1)

def NEWprob_B_die(c1, c2):
    overall_prob_B_die = 1 - prob_A_die(c1)
    d = age_at_which_death_occurred(c1, c2)
    num_B_types = sum(c1[1:])
    if d != len(c1) - 2:
        return c1[d]*overall_prob_B_die/num_B_types
    else:
        return (c1[d] + c1[d+1])*overall_prob_B_die/num_B_types

def prob_A_rep(c1, beta):
    #case no fertile B types and at least 1 A type
    if c1[-1] == 0 and c1[0] != 0:
        p = 1
        
    #case no fertile B types and no A types (in which case a young B type reproduces, no worries)
    elif c1[-1] == 0 and c1[0] == 0:
        p = 0
    
    #all other cases are covered by the following
    else:
        p = c1[0]/(c1[0] + beta*c1[-1])
    return p

def prob_B_rep(c1, beta):
    return 1 - prob_A_rep(c1, beta)   

In [28]:
def trans_prob(c1, c2, beta):
    """
    Returns probability of transitioning from state c1 (stands for 'combination 1') to state c2.
    In the events where B dies, I have to divide by the number of ages at which a B type could die.
    """
    if A_died_A_reproduced(c1, c2):
        return prob_A_die(c1)*prob_A_rep(c1, beta)
    
    elif A_died_B_reproduced(c1, c2):
        return prob_A_die(c1)*prob_B_rep(c1, beta)
    
    elif B_died_A_reproduced(c1, c2):
        return NEWprob_B_die(c1, c2)*prob_A_rep(c1, beta)
    
    elif B_died_B_reproduced(c1, c2):
        return NEWprob_B_die(c1, c2)*prob_B_rep(c1, beta)
    
#    elif B_died_A_reproduced(c1, c2):
#        if must_double_prob(c1, c2):
#            return 2*prob_B_die(c1)*prob_A_rep(c1, beta)/(len(c1) - 1)
#        else:
#            return prob_B_die(c1)*prob_A_rep(c1, beta)/(len(c1) - 1)
        
#    elif B_died_B_reproduced(c1, c2):
#        if must_double_prob(c1, c2):
#            return 2*prob_B_die(c1)*prob_B_rep(c1, beta)/(len(c1) - 1)
#        else:
#            return prob_B_die(c1)*prob_B_rep(c1, beta)/(len(c1) - 1)
        
    else:
        return 0

In [29]:
def get_state_space(N, T):
    return gen_combinations(N, T+2) # a state is a tuple (n_A, n_0, ..., n_{T-1}, n_{\ge T})

In [30]:
def build_transition_matrix(N, T, beta):
    state_space = get_state_space(N, T)  # a state is a tuple (n_A, n_0, ..., n_{T-1}, n_{\ge T})
    num_states = len(state_space)
    
    transition_matrix = zero_matrix(QQ, num_states)
    
    #the i, c2 and j, c1 looks backward, but it isn't. For prob matrices, a_{ij} = prob i <-- j
    for i, c1 in enumerate(state_space):
        for j, c2 in enumerate(state_space):
            transition_matrix[i, j] = trans_prob(c2, c1, beta)
            
    return transition_matrix

In [31]:
def get_B_fixation_prob(N, T, beta, num_iters = 1000):
    M = build_transition_matrix(N, T, beta)
    ss = get_state_space(N, T)
    v = zero_vector(len(ss))  #build state vector for N-1 A types, 1 B type w. p. 1
    v[1] = 1   #idem 
    
    for _ in range(num_iters):
        v = M*v
    
    return 1 - v[0].n()
    

def get_A_fixation_prob(N, T, beta, num_iters = 1000):
    M = build_transition_matrix(N, T, beta)
    ss = get_state_space(N, T)
    v = zero_vector(len(ss))  #build state vector for N-1 B types, 1 A type w. p. 1
    v[-1] = 1   #idem
    
    for _ in range(num_iters):
        v = M*v
    
    #kill someone uniformly at random and add someone to the category of A types.
    v = v*(N-1)/N
    v[0] = 1/N
    
    for _ in range(num_iters):
        v = M*v
        
    return v[0].n()

def get_difference_in_fixation_probs(N, T, beta, num_iters = 1000):
    return get_B_fixation_prob(N, T, beta, num_iters) - get_A_fixation_prob(N, T, beta, num_iters)

In [34]:
def find_beta_equal_fitness(N, T, num_iters = 100):
    
    def f(beta):
        return get_difference_in_fixation_probs(N, T, beta, num_iters)
    
    beta_equal = find_root(f, 1, 100)
    
    return beta_equal

In [35]:
find_beta_equal_fitness(5, 1)

1.3851120406888628

In [36]:
find_beta_equal_fitness(5, 2)

1.9924396988783089